# Imports


In [28]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
from dash.dependencies import Input, Output

# Tratando o Dataframe

In [3]:
df = pd.read_csv('testeDados.csv')
campos_vazios = df.isnull().sum()
mediana_acessos = df['acessos'].median()
data = df.drop(columns=["id_municipio", "sigla_uf", "cnpj"])

In [4]:
df['empresa'].fillna('outra', inplace=True)
df['porte_empresa'].fillna('Pequeno Porte', inplace=True)
df['tecnologia'].fillna('outra', inplace=True)
df['transmissao'].fillna('outra', inplace=True)
df['velocidade'].fillna('512kbps a 2Mbps', inplace=True)
df['acessos'].fillna(mediana_acessos, inplace=True)
df['produto'].fillna('internet', inplace=True)
df = df.dropna()

In [77]:
anos = df['ano'].unique()
df_media_por_mes = df.groupby(['mes','ano'])['acessos'].mean().reset_index()

In [86]:
app = Dash(__name__)


app.layout = html.Div(children=[
    html.H1(children='Análise dos Dados de Banda Larga', style={'font-family': 'Arial, sans-serif', 'text-align': 'center'}),
    html.Div(children='''
    1- Como está distribuído o número de acessos de acordo com os atributos apresentados?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px'}),
    dcc.Dropdown(
        id='ano-dropdown',
        options=[{'label': ano, 'value': ano} for ano in anos],  # Criando as opções do menu suspenso
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),
        html.Div(children=''' ''', style={'padding-bottom': '30px'}),

    dcc.Graph(
        id='graph-content'
    )
])

@app.callback(
    Output('graph-content', 'figure'),
    [Input('ano-dropdown', 'value')]
)
def update_graph(selected_ano):
    # Filtrar o DataFrame pelo ano selecionado
    df_ano_selecionado = df_media_por_mes[df_media_por_mes['ano'] == selected_ano]
    
    # Criar o gráfico atualizado (gráfico de barras)
    fig_ano_mes = px.bar(df_ano_selecionado, x='mes', y='acessos', title=f'Média de Acessos por Mês no Ano de: {selected_ano}', barmode='group')
    
    return fig_ano_mes

if __name__ == '__main__':
    app.run_server(debug=True)